# 第7章: データベース
[artist.json.gz](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/artist.json.gz)は，オープンな音楽データベース[MusicBrainz](https://musicbrainz.org/)の中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．

| フィールド| 型 | 内容 | 例 |
|:---:|:---:|:---:|:---:|
| id | ユニーク識別子 | 整数 | 20660|
|gid | グローバル識別子 | 文字列 | "ecf9f3a3-35e9-4c58-acaa-e707fba45060"|
|name | アーティスト名 | 文字列 | "Oasis"|
|sort_name | アーティスト名| （辞書順整列用）| 文字列 | "Oasis"
|area|活動場所|文字列	|"United Kingdom"|
|aliases|別名|辞書オブジェクトのリスト|	
|aliases[].name	|別名	|文字列	|"オアシス"|
|aliases[].sort_name	|別名（整列用）|	文字列	"オアシス"|
|begin|	活動開始日	|辞書	|
|begin.year|	活動開始年	|整数	|1991|
|begin.month|	活動開始月	|整数	|
|begin.date|	活動開始日	|整数	|
|end	|活動終了日	|辞書	|
|end.year|	活動終了年	|整数	|2009|
|end.month|	活動終了月	|整数	|8|
|end.date|	活動終了日	|整数|	28|
|tags|	タグ	|辞書オブジェクトのリスト|	
|tags[].count|	タグ付けされた回数	|整数	|1|
|tags[].value|	タグ内容	|文字列	|"rock"|
|rating	|レーティング	|辞書オブジェクト	|
|rating.count|	レーティングの投票数	|整数	|13|
|rating.value|	レーティングの値（平均値）	|整数	|86|

artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．KVSとしては，[LevelDB](http://leveldb.org/)，[Redis](http://redis.io/)，[KyotoCabinet](http://fallabs.com/kyotocabinet/)等を用いよ．ドキュメント志向型データベースとして，[MongoDB](http://www.mongodb.org/)を採用したが，[CouchDB](http://couchdb.apache.org/)や[RethinkDB](http://rethinkdb.com/)等を用いてもよい．


# 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [5]:
import redis
import json

pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
red = redis.StrictRedis(connection_pool=pool)

In [6]:
with open("artist.json",'r') as f:
    for line in f.readlines():
        data_json = json.loads(line)
        name = data_json.get("name")
        area = data_json.get("area")
        if name is not None and area is not None:
            red.sadd(name,area)

# 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [7]:
for area in red.smembers("Oasis"):
    print(area.decode("utf-8"))

United Kingdom
United States


# 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [8]:
%%time
len([key for key in red.scan_iter() if b"Japan" in red.smembers(key)])

AttributeError: module 'redis' has no attribute 'scan_iter'

In [9]:
%%time
len([key for key in red.scan_iter() if red.sismember(key,"Japan")==True])

CPU times: user 14.5 s, sys: 3.86 s, total: 18.3 s
Wall time: 25.8 s


22554

# 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

In [10]:
pool = redis.ConnectionPool(host='localhost', port=6379, db=1)
red = redis.StrictRedis(connection_pool=pool)

In [11]:
with open("artist.json",'r') as f:
    for line in f.readlines():
        data_json = json.loads(line)
        name = data_json.get("name")
        tags = data_json.get("tags")
#         print(tags)
        
        
        if name is not None and tags is not None:
            for tag in tags:
#                 print(tag)
                value,field = tag.get("count"),tag.get("value")
#                 print(name,field,value)
                red.hset(name,field,value)

In [12]:
for tag,num in red.hgetall("Oasis").items():
    print(tag.decode("utf-8"),num.decode("utf-8"))

morning glory 1
oasis 1
rock 1
britpop 3
british 4
uk 1
britannique 1
rock and indie 1
england 1
manchester 1


# 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [51]:
import json
import pymongo
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.nlp
collection = db.nlp_collection
# posts = db.posts

In [10]:
with open("artist.json",'r') as f:
    data_json = [json.loads(line) for line in f.readlines()]
#     line=f.readlines()
#     data_json = json.loads(line)

    collection.insert_many(data_json)

In [17]:

collection.create_index("name")
collection.create_index("aliases.name")
collection.create_index("tags.value")
collection.create_index("rating.value")


'rating.value_1'

In [25]:
# for result in posts.find():
#     print(result)
print(posts.find_one({"name":"Oasis"}))

{'_id': ObjectId('5dfd719f2397a460485e8781'), 'name': 'Oasis', 'aliases': ['OASIS', 'オアシス'], 'tags': ['rock', 'britpop', 'british', 'uk', 'britannique', 'rock and indie', 'england', 'manchester'], 'rating': 86}


In [97]:
# posts.find_one({"_id": post_id})

# 65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

In [24]:
for name in collection.find({"name":"Queen"}):
    print(name)

{'_id': ObjectId('5e2158c7399f5fad57a884f5'), 'name': 'Queen', 'area': 'Japan', 'gender': 'Female', 'tags': [{'count': 1, 'value': 'kamen rider w'}, {'count': 1, 'value': 'related-akb48'}], 'sort_name': 'Queen', 'ended': True, 'gid': '420ca290-76c5-41af-999e-564d7c71f1a7', 'type': 'Character', 'id': 701492, 'aliases': [{'name': 'Queen', 'sort_name': 'Queen'}]}
{'_id': ObjectId('5e2158c7399f5fad57a94ba1'), 'rating': {'count': 24, 'value': 92}, 'begin': {'date': 27, 'month': 6, 'year': 1970}, 'name': 'Queen', 'area': 'United Kingdom', 'tags': [{'count': 2, 'value': 'hard rock'}, {'count': 1, 'value': '70s'}, {'count': 1, 'value': 'queen family'}, {'count': 1, 'value': '90s'}, {'count': 1, 'value': '80s'}, {'count': 1, 'value': 'glam rock'}, {'count': 4, 'value': 'british'}, {'count': 1, 'value': 'english'}, {'count': 2, 'value': 'uk'}, {'count': 1, 'value': 'pop/rock'}, {'count': 1, 'value': 'pop-rock'}, {'count': 1, 'value': 'britannique'}, {'count': 1, 'value': 'classic pop and rock'},

# 66. 検索件数の取得
MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．

In [39]:
#!db.nlp_collection.find({"area":"Japan"}).count()
print(collection.find({"area":"Japan"}).count())

/Users/matatabinoneko/.pyenv/versions/anaconda3-5.3.1/envs/深層強化学習/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


22821


# 67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．

In [46]:
another_name = "女王"
for a in collection.find({"aliases.name":another_name}):
    print(a)

{'_id': ObjectId('5e2158c7399f5fad57a94ba1'), 'rating': {'count': 24, 'value': 92}, 'begin': {'date': 27, 'month': 6, 'year': 1970}, 'name': 'Queen', 'area': 'United Kingdom', 'tags': [{'count': 2, 'value': 'hard rock'}, {'count': 1, 'value': '70s'}, {'count': 1, 'value': 'queen family'}, {'count': 1, 'value': '90s'}, {'count': 1, 'value': '80s'}, {'count': 1, 'value': 'glam rock'}, {'count': 4, 'value': 'british'}, {'count': 1, 'value': 'english'}, {'count': 2, 'value': 'uk'}, {'count': 1, 'value': 'pop/rock'}, {'count': 1, 'value': 'pop-rock'}, {'count': 1, 'value': 'britannique'}, {'count': 1, 'value': 'classic pop and rock'}, {'count': 1, 'value': 'queen'}, {'count': 1, 'value': 'united kingdom'}, {'count': 1, 'value': 'langham 1 studio bbc'}, {'count': 1, 'value': 'kind of magic'}, {'count': 1, 'value': 'band'}, {'count': 6, 'value': 'rock'}, {'count': 1, 'value': 'platinum'}], 'sort_name': 'Queen', 'ended': True, 'gid': '0383dadf-2a4e-4d10-a46a-e9e041da8eb3', 'type': 'Group', 'id

# 68. ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [59]:
print("\n".join([a.get("name") for a in collection.find({"tags.value":"dance"}).sort("rating.value",pymongo.DESCENDING)][:10]))


SSHäuptling
Boy George
Toni Basil
Milk Inc.
digitalTRAFFIC
Bag Raiders
Syncopaths
Kristina Supergenius
MaRina
Sliptide


# 69. Webアプリケーションの作成
ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．